<a href="https://colab.research.google.com/github/ForestPearson/CS410-510-NLP-project/blob/Ray-Branch/copied_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

import nltk
from nltk import word_tokenize
#https://www.tensorflow.org/text/tutorials/text_generation

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:

path = tf.keras.utils.get_file('comedy.txt', 'https://raw.githubusercontent.com/ForestPearson/CS410-510-NLP-project/Ray-Branch/comedy.txt')

401976/401976 [==============================] - 0s 0us/step


In [4]:
# Read, then decode for py2 compat.
text = open(path, 'rb').read().decode(encoding='utf-8')


In [5]:
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

# The unique characters in the file
vocab = sorted(set(text))

print(f'{len(vocab)} unique characters')
textvocab = word_tokenize(text)

Length of text: 401976 characters
66 unique characters


In [6]:
chars = tf.strings.unicode_split(textvocab, input_encoding='UTF-8')

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[13, 15, 32], [21], [31, 15, 17, 26, 17], ..., [41, 63, 41, 65], [9],
 [17, 64, 45, 61, 54, 60]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'ACT', b'I', b'SCENE', ..., b'away', b'.', b'Exeunt'],
      dtype=object)

In [9]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids  
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
#for ids in ids_dataset.take(10):
#    print(chars_from_ids(ids).numpy().decode('utf-8'))
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

#for seq in sequences.take(1):
#  print(chars_from_ids(seq))
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"\r\nACT I\r\nSCENE I. Rousillon. The COUNT's palace.\r\nEnter BERTRAM, the COUNTESS of Rousillon, HELENA, a"
b'nd LAFEU, all in black\r\nCOUNTESS\r\nIn delivering my son from me, I bury a second husband.\r\nBERTRAM\r\nAn'
b"d I in going, madam, weep o'er my father's death\r\nanew: but I must attend his majesty's command, to\r\n"
b'whom I am now in ward, evermore in subjection.\r\nLAFEU\r\nYou shall find of the king a husband, madam; y'
b'ou,\r\nsir, a father: he that so generally is at all times\r\ngood must of necessity hold his virtue to y'


In [10]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
split_input_target(text)

("\r\nACT I\r\nSCENE I. Rousillon. The COUNT's palace.\r\nEnter BERTRAM, the COUNTESS of Rousillon, HELENA, and LAFEU, all in black\r\nCOUNTESS\r\nIn delivering my son from me, I bury a second husband.\r\nBERTRAM\r\nAnd I in going, madam, weep o'er my father's death\r\nanew: but I must attend his majesty's command, to\r\nwhom I am now in ward, evermore in subjection.\r\nLAFEU\r\nYou shall find of the king a husband, madam; you,\r\nsir, a father: he that so generally is at all times\r\ngood must of necessity hold his virtue to you; whose\r\nworthiness would stir it up where it wanted rather\r\nthan lack it where there is such abundance.\r\nCOUNTESS\r\nWhat hope is there of his majesty's amendment?\r\nLAFEU\r\nHe hath abandoned his physicians, madam; under whose\r\npractises he hath persecuted time with hope, and\r\nfinds no other advantage in the process but only the\r\nlosing of hope by time.\r\nCOUNTESS\r\nThis young gentlewoman had a father,--O, that\r\n'had'! how sad a passage 'tis!

In [11]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"\r\nACT I\r\nSCENE I. Rousillon. The COUNT's palace.\r\nEnter BERTRAM, the COUNTESS of Rousillon, HELENA, "
Target: b"\nACT I\r\nSCENE I. Rousillon. The COUNT's palace.\r\nEnter BERTRAM, the COUNTESS of Rousillon, HELENA, a"


In [12]:
# Batch size
BATCH_SIZE = 128#from64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
dataset


<PrefetchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int64, name=None), TensorSpec(shape=(128, 100), dtype=tf.int64, name=None))>

In [13]:


# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [14]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 67) # (batch_size, sequence_length, vocab_size)


In [16]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  17152     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  68675     
                                                                 
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [17]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [18]:
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))
# Directory where the checkpoints will be saved
dir = './data/epochs'
#File names
fileName = os.path.join(dir, "ckpt_{epoch}")
results = tf.keras.callbacks.ModelCheckpoint(filepath=fileName,save_weights_only=True)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    #st

(128, 100, 67) # (batch_size, sequence_length, vocab_size)


In [21]:
model
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  17152     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  68675     
                                                                 
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [23]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'her, the affection\r\nof nobleness which nature shows above her breeding,\r\nand many other evidences pr'

Next Char Predictions:
 b"FWPyVg-CeJpprqetvnrk\rlV-'\nHR[,[UNK]RkUQqodIXp\nctY[UNK]Xx;m'ytEEqlgW YP[UNK]]l!mm]-\r-wpEWd-tA! LrTI.D!eOv\rnEwrTag"


In [24]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [25]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (128, 100, 67)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.2036047, shape=(), dtype=float32)


In [26]:
tf.exp(example_batch_mean_loss).numpy()

66.927155

In [27]:
model.compile(optimizer='adam', loss=loss)

In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [29]:
EPOCHS = 20

In [30]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
31/31 [==============================] - 5s 96ms/step - loss: 3.8111
Epoch 2/20
31/31 [==============================] - 3s 93ms/step - loss: 2.8482
Epoch 3/20
31/31 [==============================] - 3s 95ms/step - loss: 2.4324
Epoch 4/20
31/31 [==============================] - 3s 95ms/step - loss: 2.2889
Epoch 5/20
31/31 [==============================] - 3s 95ms/step - loss: 2.2001
Epoch 6/20
31/31 [==============================] - 3s 94ms/step - loss: 2.1159
Epoch 7/20
31/31 [==============================] - 3s 95ms/step - loss: 2.0266
Epoch 8/20
31/31 [==============================] - 3s 95ms/step - loss: 1.9363
Epoch 9/20
31/31 [==============================] - 3s 97ms/step - loss: 1.8513
Epoch 10/20
31/31 [==============================] - 3s 96ms/step - loss: 1.7778
Epoch 11/20
31/31 [==============================] - 4s 96ms/step - loss: 1.7129
Epoch 12/20
31/31 [==============================] - 3s 98ms/step - loss: 1.6570
Epoch 13/20
31/31 [==================

In [31]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [32]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [34]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO: Forself,
Shall she is awam; who flay be so floved: ho is mean
To me hus quay be nobeding:
In that of horoun by it an ouths am butweddable.
PAULINA
Good Oreviones, reportune that
Ents they hish lack, whyse Door menally. Be count
As allly thrive a one thou hast, now: weary, sir, now loven and
dived, stirling in ot fies: the honour in'ble away,
Ap the wife them is her songure please 'lack.
ORDANA and WIONTESE I shait of: an eye: helf you do couns
of the father. Shephing that she mistless that we
You'rl meaning place was vice; or aught.
Steponders and to trualy thement infect, eys
The incessevires for ou'tr,
You she, shall new an heaveng. Benite
I live hur bather, I'll gold! I need not?
Shepherd
I am; anainst
as great carteful man.
Clown
Our fortune's nade, as deailst that ends a retwert.
LEONTES
[Aside to her, dawald abtake to dued?
Wiscominity he her newer youth, fear
As evcluaness credif to his eye.
ORLANDO
I shall be their butger.
ORLANDO
Or your-grean for 

____________________

In [35]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')